# Title

## Introduction

1. background on the topic
2. **state the question you will try to answer**
    - one or more random variables
    - spread across two or more categories that are interesting to compare
    - of the response variable, identify one location parameter (mean, median, quantile, etc.) and one scale parameter (standard deviation, inter-quartile range, etc.) that would be useful in answering your question. 
3. Identify and describe the dataset that will be used to answer the question. 
4. at least two scientific publications

## Preliminary Results

- Demonstrate that the dataset can be read from the web into R.
- Clean and wrangle your data into a tidy format.
- Plot the relevant raw data, tailoring your plot in a way that addresses your question.
- Compute estimates of the parameter you identified across your groups. 
- Present this in a table. If relevant, include these estimates in your plot.

In [3]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [5]:
raw_leaf <- read_csv("data/leaf.csv")
spec(raw_leaf)

New names:
* `1` -> `1...1`
* `1` -> `1...2`
* `1` -> `1...7`

Rows: 339 Columns: 16

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (16): 1...1, 1...2, 0.72694, 1.4742, 0.32396, 0.98535, 1...7, 0.83592, 0...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



cols(
  `1...1` = col_double(),
  `1...2` = col_double(),
  `0.72694` = col_double(),
  `1.4742` = col_double(),
  `0.32396` = col_double(),
  `0.98535` = col_double(),
  `1...7` = col_double(),
  `0.83592` = col_double(),
  `0.0046566` = col_double(),
  `0.0039465` = col_double(),
  `0.04779` = col_double(),
  `0.12795` = col_double(),
  `0.016108` = col_double(),
  `0.0052323` = col_double(),
  `0.00027477` = col_double(),
  `1.1756` = col_double()
)

## Methods: Plan

## References